In [3]:
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import re
from multiprocessing import Pool, Process
import os


In [47]:
def getAllUrl(url, url_base, urls, errors):
    
    page = requests.get(url)

    if page.status_code == 200:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        for anchor in soup.find_all("a"):
            if "href" in anchor.attrs:
                if anchor["href"] == "JavaScript:void(0);" and anchor["class"] == "lien-page":
                    absolute_url == url+"?pR="+anchor["page"]
                # Manque "?pI=" pour les interviews
                absolute_url = urljoin(url_base, anchor["href"])
                if "download" not in absolute_url and absolute_url not in urls:
                    if re.search(url_base, absolute_url):
                        urls.append(absolute_url)

        # Export des pages html ?

    else:
        print(" ", page.status_code)
        errors.append(url)



In [24]:
def read_urls(EXPORT_FILE):
    with open(EXPORT_FILE, "r") as f:
        urls = f.read().split("\n") 
    print(len(urls))
    return(urls)

In [31]:
def get_all_page(urls):

    pattern = "https://www.millenaire3.com/m3_search?page=1&sort_by=0&SearchText=&modeRecherche=1&titre=&auteurId=&auteurText=&100%25=&100%25_new_value=true&date_debut=&date_debut_mobile=&date_fin=&date_fin_mobile=&identifiers=cactualite_a_la_une%2CrArticle%2Ccdossier%2Cr%C3%89tude%2Ccinterview%2Ccressource_generique%2CrTexte+d%27auteur&rubriques="
    html = requests.get(pattern)
    soup = BeautifulSoup(html.content, "html.parser")
    last_page = soup.find("a", {"title" : "Dernière page"})["page"]
    print(last_page)
    for i in range(1, int(last_page)+1):
        urls.append(pattern.replace("m3_search?page=1", f"m3_search?page={i}"))
    return(urls)

In [48]:

EXPORT_FILE = "M3urls3.txt"

if __name__ == "__main__":
    
    url_base = 'https://www.millenaire3.com'
    errors = []
    urls = get_all_page(urls = [])
    # if EXPORT_FILE in os.listdir():
    #     urls = read_urls(EXPORT_FILE)
    # else:
    #     urls = [url_base]

    # url_base = "https://www.millenaire3.com/dossiers/cycle-veille-m3-energie-quels-nouveaux-modes-de-vie-dans-un-monde-plus-sobre/veille-m3-l-ordre-electrique-l-architecture-des-infrastructures-energetiques-matrice-de-nos-imaginaires-sociaux"   
    i = 0 # On peut changer la valeur de départ
    while i <= len(urls)-1:
        getAllUrl(urls[i], url_base, urls, errors)
        i += 1
        print(f"\r{i} / {len(urls)} : {urls[i-1]}", end = "")

with open(EXPORT_FILE, "w", encoding="utf-8") as f:
    for url in urls:
        f.write(url+"\n")

168
40 / 591 : https://www.millenaire3.com/m3_search?page=40&sort_by=0&SearchText=&modeRecherche=1&titre=&auteurId=&auteurText=&100%25=&100%25_new_value=true&date_debut=&date_debut_mobile=&date_fin=&date_fin_mobile=&identifiers=cactualite_a_la_une%2CrArticle%2Ccdossier%2Cr%C3%89tude%2Ccinterview%2Ccressource_generique%2CrTexte+d%27auteur&rubriques=

In [34]:
urls = read_urls(EXPORT_FILE)

12367
